# Podcast Episode Erstellen

### imports

In [11]:
import sys
sys.path.append(".")
from audio_transcription.faster_whisper_word_level import transcribe
from Episodes_Downloader.episodes_downloader import get_names_and_urls_all_episodes, download_and_save_mp3_in_dir
import os
import pandas as pd
from pydub import AudioSegment
from pydub.playback import play
from playsound import playsound
import IPython 
import io
from db_connect import db_get_df,db_save_df
# from segment_ranking.rank_segments import get_most_similar_documents_Llama2, get_most_similar_documents_tf_idf
# from Audio_segmentation.split_audio import produce_snippets
# from Audio_segmentation.concat_audio import produce_audio
from tqdm import tqdm

### Episoden URLs laden

In [ ]:
titles, audio_urls = get_names_and_urls_all_episodes()
print(titles)

In [ ]:
print(len(titles))

In [ ]:
df = pd.DataFrame({"title": titles, "download_url": audio_urls, "transcript": None})
df.dtypes

In [ ]:
db_save_df(df, "transcripts_all")

### Alle mp3s herunterladen

In [ ]:
path = "/Volumes/Samsung_T5/Podcast_Episoden"
os.listdir(path)

In [ ]:
df = db_get_df("transcripts_all")

for i, row in tqdm(df.iterrows()):
    title = row["download_url"].split("/")[-1]
    if title in os.listdir(path):
        continue
    download_and_save_mp3_in_dir(row["download_url"], path, title)


In [ ]:
df.loc[179, "download_url"]

### Transkription auf Word level Ebene

In [ ]:
df = transcribe("/Users/br/Projects/Bachelorarbeit/data/Episode_audio_files/jonathan swift gullivers reisen 2.mp3")
# db_save_df(df,"transcript_word_level")

In [ ]:
db_save_df(df, "transcript_gulliver_word_level")

### Debug db

In [ ]:
import sqlite3
import pandas as pd

con = sqlite3.connect("/Volumes/Samsung_T5/Backup/transcripts.sqlite")
df = pd.read_sql_query(f"SELECT * FROM transcript_word_level_2237", con)
con.close()

In [ ]:
len(df)

In [ ]:
print(df.dtypes)
print(df["filename"].drop_duplicates(ignore_index=True))

Die Namen der Filenames ändern

In [ ]:
df['filename'] = df['filename'].str.replace('/nfs/scratch/students/neumannvi84434/Podcast_Episoden/', '')

In [ ]:
with sqlite3.connect("/Volumes/Samsung_T5/Backup/transcripts.sqlite") as con:
    df.to_sql("transcript_word_level_2237", con, index=False, if_exists='replace')

In [ ]:
db_save_df(df, "transcript_word_level_2237")

##### setence id add

In [ ]:
df =db_get_df("transcript_sentences")

In [ ]:
df['segment_id'] = df.groupby('filename').cumcount()

In [ ]:
df

In [ ]:
db_save_df(df, "transcript_sentences")

### Lemmatisierung mit spacy

In [ ]:
import spacy

In [ ]:
def lemmatize_german_sentence(input_sentence, nlp):
    doc = nlp(input_sentence)
    lemmatized_words = []
    for token in doc:
        lemmatized_words.append(token.lemma_)
    return lemmatized_words

In [ ]:
df = db_get_df("transcript_sentences")
df.dtypes

In [ ]:
nlp = spacy.load("de_core_news_md")
all_sentences = []
for sentence in tqdm(df["sentence"]):
    all_sentences.append(lemmatize_german_sentence(sentence, nlp))

In [ ]:
all_sentences[3000]

In [ ]:
len(all_sentences)

In [ ]:
all_sentences_full = [" ".join(sentence) for sentence in all_sentences]

In [ ]:
len(all_sentences_full)

In [ ]:
df["sentence_lemmatized"] = all_sentences_full

In [ ]:
db_save_df(df, "sentences_lemmatized")

In [ ]:
all_sentences_full[4000]

### Kompositatrennung mit pyphen

In [ ]:
import pyphen

# Erstellen Sie ein Wörterbuch für die deutsche Sprache
deutsche_worte = pyphen.Pyphen(lang='de_DE')

# Set zum Speichern der einzelnen Wörter initialisieren
einzelwoerter_set = set()

# Pfad zur Textdatei
dateipfad = '/Users/br/Projects/Bachelorarbeit/scripts/Embedding_creation/vocabulary.txt'

# Textdatei öffnen und lesen
with open(dateipfad, 'r', encoding='utf-8') as datei:
    for zeile in datei:
        # Wörter in der Zeile aufteilen (z.B. Leerzeichen als Trennzeichen verwenden)
        woerter = zeile.strip().split()
        
        for wort in woerter:
            # Wort trennen
            getrenntes_wort = deutsche_worte.inserted(wort)
            
            # Trennungsergebnis in einzelne Wörter aufteilen
            einzelwoerter = getrenntes_wort.split('-')
            
            # Einzelne Wörter zum Set hinzufügen
            einzelwoerter_set.update(einzelwoerter)

# Ausgabe der einzelnen Wörter im Set
for wort in einzelwoerter_set:
    print(wort)


In [ ]:
len(einzelwoerter_set)

In [ ]:
from german_compound_splitter import comp_split
from tqdm import tqdm

einzelwoerter_set = set()
dateipfad = '/Users/br/Projects/Bachelorarbeit/scripts/Embedding_creation/vocabulary.txt'
input_file = '/Users/br/Projects/Bachelorarbeit/german.dic'
ahocs = comp_split.read_dictionary_from_file(input_file)

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False
    

with open(dateipfad, 'r', encoding='utf-8') as datei:

    for zeile in tqdm(datei):
        wort = zeile.strip()
        if is_number(wort):
            continue
        try:
            dissection = comp_split.dissect(wort, ahocs, make_singular=True)
        except:
            print(wort)
            dissection = ""
        einzelwoerter_set.update(wort)

print(len(einzelwoerter_set))
print(einzelwoerter_set)

In [ ]:
print(len(einzelwoerter_set))

In [ ]:
df = db_get_df("sentences_lemmatized")

In [ ]:
word = "seife"
occurences = []
for sentence in tqdm(df["sentence"]):
    if word in sentence.lower():
        occurences.append(sentence)
print(occurences)

In [ ]:
df = db_get_df("transcript_sentences")

In [ ]:
word = "chiemgau"
occurences = []
for sentence in tqdm(df["sentence"]):
    if word in sentence.lower():
        occurences.append(sentence)
print(occurences)

### Satzbildung durch Whisper Punkte

Zunächst erstmal mit Whispers Punkten

In [ ]:
df = db_get_df("transcript_word_level_2237")

In [ ]:
df

In [ ]:
one_sentence = []
sentence_dict = [] 
filenames = df["filename"].drop_duplicates(ignore_index=True)

for filename in tqdm(filenames):
    word_entries = df[df['filename'] == filename]
    start = -1
    end = -1
    for index, word_entry in word_entries.iterrows(): 
        word = word_entry["word"]
        
        if not one_sentence: ##start
            start = word_entry["start"]
        one_sentence.append(word.strip())
        if "." in word:
            end = word_entry["end"]
            sentence = " ".join(one_sentence)
            sentence_dict.append({"filename": filename, "sentence": sentence, "start": start, "end":end})
            one_sentence = []

df_sentences = pd.DataFrame(sentence_dict)

In [ ]:
db_save_df(df_sentences, "transcript_sentences")

In [ ]:
df_sentences

In [ ]:
df

### Demonstration

In [ ]:
df = db_get_df(table="transcript_sentences")
df.dtypes

In [ ]:
userInputText = "Zugspitze wandern"
userInput_segment_count = 7
best_fitting = get_most_similar_documents_tf_idf(userInputText, userInput_segment_count)
print(best_fitting["sentence"].to_markdown())

produce_snippets()
produce_audio()


In [ ]:
IPython.display.Audio("/Users/br/Projects/Bachelorarbeit/scripts/server/audio/concatenated_audio.mp3")

## umwandelen aller MP3s in WAVs

In [ ]:
from pydub import AudioSegment
import os
from tqdm import tqdm

def convert_mp3_to_wav(source_dir, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    
    for filename in tqdm(os.listdir(source_dir)):
        if filename.endswith('.mp3') and filename.replace('.mp3', '.wav') not in os.listdir(target_dir) :
            mp3_path = os.path.join(source_dir, filename)
            wav_path = os.path.join(target_dir, filename.replace('.mp3', '.wav'))
            
            
            audio = AudioSegment.from_mp3(mp3_path)
            
            audio.export(wav_path, format="wav")
            print(f"Converted {filename} to WAV and saved to {wav_path}")

# Example usage
source_directory = '/Volumes/Samsung_T5/Podcast_Episoden'
target_directory = '/Volumes/Samsung_T5/Podcast_Episoden_Wav'
convert_mp3_to_wav(source_directory, target_directory)


## Random

In [9]:
from db_connect import db_get_df,db_save_df
from tqdm import tqdm
import os

In [15]:
df = db_get_df("transcript_word_level_2237")

In [16]:
len(df)

5432229

In [17]:
os.listdir(directory)[0]

'baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3'

In [19]:
df.head()

,filename,index,word,start,end
0,/nfs/scratch/students/neumannvi84434/Podcast_E...,0.0,Verfolgt,11.70,12.60
1,/nfs/scratch/students/neumannvi84434/Podcast_E...,1.0,und,12.60,12.88
2,/nfs/scratch/students/neumannvi84434/Podcast_E...,2.0,ausgegrenzt.,12.88,13.82
3,/nfs/scratch/students/neumannvi84434/Podcast_E...,3.0,Der,14.08,14.40
4,/nfs/scratch/students/neumannvi84434/Podcast_E...,4.0,niederländische,14.40,15.04


In [18]:
df[df["filename"] == "baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3" ]

,filename,index,word,start,end
5429585,baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3,0.0,Verfolgt,11.70,12.60
5429586,baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3,1.0,und,12.60,12.88
5429587,baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3,2.0,ausgegrenzt.,12.88,13.82
5429588,baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3,3.0,Der,14.08,14.40
5429589,baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3,4.0,niederländische,14.40,15.04
...,...,...,...,...,...
5432224,baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3,2.0,Denkers,1388.84,1389.24
5432225,baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3,3.0,in,1389.24,1389.66
5432226,baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3,4.0,aktuellen,1389.66,1390.12
5432227,baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3,0.0,Übersetzungen,1390.12,1390.74


In [11]:
directory = "/nfs/scratch/students/neumannvi84434/Podcast_Episoden/"
list = df["filename"].to_list()
for filename in tqdm(os.listdir(directory)):    
    path = os.path.join(directory, filename)
    if filename.endswith(".mp3") and filename not in list:
        print(filename)

  7%|▋         | 167/2239 [00:00<00:01, 1656.03it/s]

attila-und-die-hunnen-das-kriegervolk-aus-der-steppe-2.mp3
oasen-inseln-der-wueste.mp3
der-versailler-vertrag-ende-und-anfang-einer-katastrophe.mp3
pierre-auguste-renoir-malerei-voll-licht-und-luft.mp3
abraham-lincoln-albtraum-buergerkrieg-und-traum-von-freiheit.mp3
hindenburg-ein-deutscher-mythos.mp3
die-milz-filter-fuer-unser-blut.mp3
eindeutig-klimawandel-was-forscher-wissen-und-was-schon-geschieht.mp3
methan-das-unterschaetzte-klimagas.mp3
die-tabakpflanze-gefaehrliche-verfuehrerin.mp3
spontanitaet-die-grosse-herausforderung.mp3
beowulf-archetyp-und-monstertoeter.mp3
hexer-und-heiler-eine-schattengeschichte-der-deutschen.mp3
philipp-reis-erfinder-des-telefons.mp3
gruene-kraftstoffe-heilsbringer-fuer-verbrennungsmotoren.mp3
im-grenzland-die-frontier-in-den-usa-2.mp3
frauenstimmrecht-in-der-schweiz-der-lange-weg-zur-wahlurne.mp3
therese-giehse-ein-starker-charakter-1.mp3
fuerstbischof-julius-echter-alle-macht-dem-glauben.mp3
martin-luther-ein-moench-erschuettert-das-abendland.mp3
als

 21%|██        | 467/2239 [00:00<00:01, 1299.43it/s]

der-hitlerputsch-anfang-vom-ende-der-demokratie.mp3
we-shall-overcome-der-civil-rights-act-von-1964.mp3
das-ohr-wie-hoeren-unser-leben-praegt-1.mp3
hedwig-courths-mahler-was-heisst-hier-kitsch.mp3
die-zirbelkiefer-ein-baum-des-hochgebirges.mp3
der-hering-silber-des-meeres.mp3
tibets-weisheit-fuehrer-auf-dem-weg-zum-erwachen.mp3
marsmaennchen-realitaet-fiktion-goetter-der-gottlosen.mp3
die-geschichte-der-eisenbahn-in-deutschland-vom-adler-zum-ice.mp3
die-kultur-der-puenktlichkeit-typisch-deutsch.mp3
wieviel-salz-darf-in-die-suppe-forschungsstreit-um-das-weisse-mineral.mp3
versklavt-im-mittelmeer-als-christen-und-moslems-sich-jagten.mp3
der-schrebergarten-idylle-im-kleinformat.mp3
otl-aicher-gestaltung-mit-haltung.mp3
artemisia-gentileschi-malerin-des-barock.mp3
alan-turing-computerpionier-codeknacker-visionaer.mp3
die-gebaermutter-der-uterus-viel-mehr-als-ein-teil-des-geschlechtsorgans.mp3
das-fenster-geschichte-mit-durchblick.mp3
das-dunkle-zeitalter-des-kosmos-bevor-das-weltall-leucht

 37%|███▋      | 826/2239 [00:00<00:00, 1577.81it/s]

...

die-deutsch-amerikanische-freundschaft-eine-wechselhafte-beziehung.mp3
die-brooklyn-bridge-die-unglaubliche-geschichte-ihrer-entstehung-1.mp3
seesterne-bizarre-schoenheiten-der-meere.mp3
tiere-verstehen-zwischen-deutung-und-forschung.mp3
vom-bettelruf-zur-bravour-arie-warum-voegel-singen-1.mp3
die-geschichte-der-imkerei-das-geschaeft-mit-der-biene.mp3
hoehenluft-ganz-schoen-gut-ganz-schoen-schlecht-1.mp3
das-faultier-langsamkeit-als-ueberlebenstrick-alles-natur.mp3
annette-von-droste-huelshoff-die-judenbuche.mp3
die-mandeln-besser-als-ihr-ruf.mp3
die-wannseekonferenz-die-absprache-zum-voelkermord.mp3
der-thunfisch-der-bedrohte-jaeger-der-meere.mp3
der-schweinswal-opfer-einer-verfehlten-meerespolitik.mp3
berggeister-alte-kulte-und-hohe-berge.mp3
kreislaufwirtschaft-wirtschaften-ohne-ressourcenverschwendung.mp3
die-geschichte-der-wall-street-wie-maechtig-ist-die-geldfabrik.mp3
multiple-persoenlichkeit-ich-bin-viele.mp3
der-chinesische-weise-menzius-lehrer-des-mitgefuehls-1.mp3
eichhoernc

 86%|████████▌ | 1922/2239 [00:01<00:00, 1798.37it/s]

runzelige-rinden-und-bunte-borken-die-haut-der-baeume.mp3
die-reliquienraeuber-von-bari-nikolaus-in-geschichte-und-legende.mp3
flexibel-im-beruf-ungezwungen-ueberarbeitet-1.mp3
rastafari-leben-im-hier-und-jetzt.mp3
die-wiener-werkstaette-und-die-frauen-textildesign-mode-und-keramik.mp3
der-feuersalamander-waldlurch-mit-vorliebe-fuer-klares-wasser.mp3
dornroeschen-das-erwachen-der-sexualitaet.mp3
diagnose-durch-roentgen-eine-medizinische-revolution-1.mp3
friedrich-duerrenmatt-theater-als-gesellschaftskritik-1.mp3
kriegsenkel-das-besondere-erbe-dieser-generation.mp3
geschichte-des-sparens-die-hoffnung-auf-der-hohen-kante.mp3
sammellust-sammelwut-von-der-jagd-nach-den-dingen.mp3
spuk-und-trug-eine-erschreckende-geschichte-der-poltergeister.mp3
das-bessere-ist-der-feind-des-guten-voltaire.mp3
humanismus-renaissance-reformation-aufbruch-in-die-neuzeit.mp3
was-uns-aufrecht-haelt-die-wirbelsaeule-ein-erfolgsmodell.mp3
das-jahr-1000-als-die-globalisierung-begann-1.mp3
die-staendegesellschaft-i

100%|██████████| 2239/2239 [00:01<00:00, 1685.91it/s]

kulturgeschichte-des-haushalts-mehr-als-kochen-und-putzen-1.mp3
die-weisheit-der-kabbala-zwischen-alltag-und-ekstase-1.mp3
rajneesh-bhagwan-osho-der-guru-der-70er-1.mp3
die-kueche-altbayerns-dienstag-nudeltag-mittwoch-strudeltag-1.mp3
das-vielvoelkerreich-im-osten-russland-und-die-sowjetunion.mp3
karl-wolfskehl-der-zeus-von-schwabing-muss-ins-exil.mp3
oriana-fallaci-kompromisslos-stolz-und-unbequem-1.mp3
die-sogenannten-menschenrassen-die-konstruktion-einer-ungleichheit.mp3
zuckerruebe-suess-und-voller-geschichte.mp3
immanuel-kant-der-weltweise-aus-koenigsberg.mp3
das-vorbild-mit-gutem-beispiel-voran.mp3
hofmannsthals-jedermann-salzburg-forever.mp3
wie-wir-menschen-wurden-kochen-nahrung-und-gehirn.mp3
besonnenheit-die-kraft-der-mitte.mp3
maerchen-mutmacher-und-entwicklungshelfer.mp3
zwischen-bierkeller-und-salon-hitler-in-muenchen-vor-1933.mp3
die-geschichte-des-aufzugs-wie-der-fahrstuhl-die-welt-veraenderte.mp3
bayern-und-napoleon-der-grosse-schritt-nach-vorne-1.mp3
trauma-wie-umgehen

In [3]:
df["filename"].head()

0    baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
1    baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
2    baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
3    baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
4    baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
Name: filename, dtype: object

In [6]:
df["filename"]

0          baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
1          baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
2          baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
3          baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
4          baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
                                 ...                        
5429580         autofiktion-wer-schreibt-hier-eigentlich.mp3
5429581         autofiktion-wer-schreibt-hier-eigentlich.mp3
5429582         autofiktion-wer-schreibt-hier-eigentlich.mp3
5429583         autofiktion-wer-schreibt-hier-eigentlich.mp3
5429584         autofiktion-wer-schreibt-hier-eigentlich.mp3
Name: filename, Length: 5429585, dtype: object

In [3]:
directory = "/nfs/scratch/students/neumannvi84434/Podcast_Episoden/"
len(os.listdir(directory))

2239

In [19]:
os.listdir(directory)[1948]

'gluehwuermchen-mit-licht-auf-brautschau-1.mp3'

In [13]:
list = df["filename"].to_list()
for filename in tqdm(os.listdir(directory)):
    if filename not in list:
        print(filename)

  0%|          | 0/2239 [00:00<?, ?it/s]

  6%|▋         | 141/2239 [00:00<00:08, 237.29it/s]

Die Geschichte der Street Art - Schmierereien und  oder Kunst?


 40%|███▉      | 892/2239 [00:18<01:02, 21.55it/s] 

Romantik - Zwischen Schwärmerei und Freiheitsstreben


...

100%|█████████▉| 2233/2239 [01:51<00:00, 11.44it/s]

die-violine-die-geschichte-eines-instruments.mp3
hagia-sophia-kirche-moschee-museum-moschee.mp3
sperma-viel-mehr-als-wunder-des-lebens.mp3
maria-callas-operndiva-mit-kultstatus.mp3
was-die-bibel-ueber-die-evolution-verraet-das-tagebuch-der-menschheit.mp3
die-metapher-vom-umgang-mit-sprachbildern.mp3


100%|██████████| 2239/2239 [01:51<00:00, 11.47it/s]

von-mondholz-und-vollmondbier-was-der-erdtrabant-tatsaechlich-bewirkt.mp3
dynamit-ein-sprengstoff-macht-geschichte.mp3
william-wyler-ben-hur-und-die-oscars.mp3
francisco-de-goya-hofmaler-und-sozialkritiker.mp3
der-oberoesterreichische-bauernkrieg-bayern-gegen-protestanten.mp3
der-ganges-lebensader-und-heiligtum.mp3


100%|██████████| 2239/2239 [01:51<00:00, 20.05it/s]
